# Install modules

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import os
from tensorflow.keras.utils import to_categorical

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM
from keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall, AUC
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU device 0

# Get data in right format

In [20]:
df = pd.read_csv("preprocesseddata.csv",index_col="datetime")

#splitting into target and input variables
X = df.drop(['classification_region1', 'classification_region2', 'classification_region3'], axis=1)
X = X.drop(X.columns[:30], axis=1)
display(X)
y1 = df[['classification_region1']]
y2 = df[['classification_region2']]
y3 = df[['classification_region3']]

# Define the number of splits for the TimeSeriesSplit
n_splits = 5

# Define the TimeSeriesSplit object
tscv = TimeSeriesSplit(n_splits=n_splits)

,day,month,weekday,dayofyear,weekofyear,quarter,3.feed_5975.PM2_5_region_2,3.feed_11067.CO_PPB..3.feed_43.CO_PPB_region_2,3.feed_11067.NO2_PPB..3.feed_43.NO2_PPB_region_2,3.feed_11067.NOX_PPB..3.feed_43.NOX_PPB_region_2,...,3.feed_26.PM10B_UG_M3..3.feed_59665.PM10_640_UG_M3_region_0,3.feed_24.PM10_UG_M3_region_1,3.feed_23.PM10_UG_M3_region_0,3.feed_23.CO_PPM..3.feed_23.CO_PPB_region_0,3.feed_1.PM25B_UG_M3..3.feed_1.PM25T_UG_M3..3.feed_1.PM25_640_UG_M3_region_0,3.feed_1.SO2_PPM_region_0,3.feed_1.H2S_PPM_region_0,3.feed_1.SIGTHETA_DEG_region_0,3.feed_1.SONICWD_DEG_region_0,3.feed_1.SONICWS_MPH_region_0
datetime,,,,,,,,,,,,,,,,,,,,,
2016-11-01 00:00:00+00:00,-1.666416,1.318167,-1.000146,1.187559,1.174977,1.365179,0.227266,-0.951510,0.178303,-0.234613,...,0.351022,0.429520,1.289698,0.910414,0.222308,1.618415,0.021854,1.132898,-1.052163,-0.996418
2016-11-02 00:00:00+00:00,-1.552666,1.318167,-0.499782,1.196903,1.174977,1.365179,1.991253,-0.023215,0.613108,-0.068115,...,1.187876,1.669449,3.497008,2.131927,1.307414,0.622728,-0.391097,-0.365414,-0.919044,-0.574417
2016-11-03 00:00:00+00:00,-1.438916,1.318167,0.000582,1.206247,1.174977,1.365179,0.518567,-0.088993,0.366431,0.140433,...,-0.687044,-0.102437,0.204481,-0.616477,-0.066231,-0.463475,-0.391097,-0.321578,0.126188,0.220516
2016-11-04 00:00:00+00:00,-1.325166,1.318167,0.500946,1.215591,1.174977,1.365179,-0.865111,0.618985,0.212169,1.398540,...,-1.350126,-0.947310,-0.938685,-0.952393,-1.604286,-0.463475,-0.391097,-0.333755,1.659523,-0.375684
2016-11-05 00:00:00+00:00,-1.211416,1.318167,1.001310,1.224936,1.174977,1.365179,1.594761,2.950506,0.977215,1.996340,...,-0.110849,0.363025,-0.185355,0.421809,0.177096,-0.017356,-0.391097,-0.733751,-0.447140,-0.797685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-12 00:00:00+00:00,-0.415165,0.175202,-1.500510,0.131667,0.129641,0.478612,-0.768011,-0.630344,-1.221109,-1.046075,...,-0.622108,-0.147844,-0.358721,-1.000000,-0.220777,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2021-07-13 00:00:00+00:00,-0.301415,0.175202,-1.000146,0.141011,0.129641,0.478612,-0.519631,-0.850983,-1.240051,-1.139837,...,-0.267375,0.077490,0.042317,-1.000000,-0.320245,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2021-07-14 00:00:00+00:00,-0.187665,0.175202,-0.499782,0.150355,0.129641,0.478612,-0.548800,-0.531101,-1.179016,-1.030732,...,-0.286451,-0.286615,-0.074726,-1.000000,-0.265989,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000


# Build and train model using timeseries split

In [21]:
# Define the parameter grid for hyperparameter search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Split the data into training and testing sets using TimeSeriesSplit
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y1_train, y1_test = y1.iloc[train_index], y1.iloc[test_index]
    y2_train, y2_test = y2.iloc[train_index], y2.iloc[test_index]
    y3_train, y3_test = y3.iloc[train_index], y3.iloc[test_index]

    # Perform hyperparameter search
    grid_search = GridSearchCV(
        estimator=RandomForestClassifier(random_state=42),
        param_grid=param_grid,
        scoring='f1_macro',
        n_jobs=-1,
        cv=3
    )
    grid_search.fit(X_train, y1_train)
    rf1 = grid_search.best_estimator_

    grid_search.fit(X_train, y2_train)
    rf2 = grid_search.best_estimator_

    grid_search.fit(X_train, y3_train)
    rf3 = grid_search.best_estimator_

    # Make predictions
    y1_pred = rf1.predict(X_test)
    y2_pred = rf2.predict(X_test)
    y3_pred = rf3.predict(X_test)

    # Calculate F1 scores
    f1_score1 = f1_score(y1_test, y1_pred)
    f1_score2 = f1_score(y2_test, y2_pred)
    f1_score3 = f1_score(y3_test, y3_pred)

    # Print final F1 scores
    print("F1 score for classification_region1: {:.2f}".format(f1_score1))
    print("F1 score for classification_region2: {:.2f}".format(f1_score2))
    print("F1 score for classification_region3: {:.2f}".format(f1_score3))

C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  

F1 score for classification_region1: 0.04
F1 score for classification_region2: 0.00
F1 score for classification_region3: 0.08


C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


F1 score for classification_region1: 0.16
F1 score for classification_region2: 0.00
F1 score for classification_region3: 0.23


C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


F1 score for classification_region1: 0.07
F1 score for classification_region2: 0.00
F1 score for classification_region3: 0.15


C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


F1 score for classification_region1: 0.11
F1 score for classification_region2: 0.08
F1 score for classification_region3: 0.27


C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\Rick\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


F1 score for classification_region1: 0.41
F1 score for classification_region2: 0.05
F1 score for classification_region3: 0.55


# Evaluate model using test data

In [23]:
df = pd.read_csv("test_preprocesseddata.csv",index_col="datetime")

X = df.drop(['classification_region1', 'classification_region2', 'classification_region3'], axis=1)
X = X.drop(X.columns[:30], axis=1)

y1 = df[['classification_region1']]
y2 = df[['classification_region2']]
y3 = df[['classification_region3']]


# Make predictions
y1_pred = rf1.predict(X)
y2_pred = rf2.predict(X)
y3_pred = rf3.predict(X)

# Calculate F1 scores
f1_score1 = f1_score(y1, y1_pred)
f1_score2 = f1_score(y2, y2_pred)
f1_score3 = f1_score(y3, y3_pred)

# Calculate accuracy
accuracy1 = accuracy_score(y1, y1_pred)
accuracy2 = accuracy_score(y2, y2_pred)
accuracy3 = accuracy_score(y3, y3_pred)

# Calculate roc_auc_score (Assuming binary classification)
roc_auc1 = roc_auc_score(y1, y1_pred)
roc_auc2 = roc_auc_score(y2, y2_pred)
roc_auc3 = roc_auc_score(y3, y3_pred)

# Print final scores for test set
print("Accuracy for classification_region1: {:.2f}".format(accuracy1))
print("F1 score for classification_region1: {:.2f}".format(f1_score1))
print("ROC AUC score for classification_region1: {:.2f}".format(roc_auc1))

print("\nAccuracy for classification_region2: {:.2f}".format(accuracy2))
print("F1 score for classification_region2: {:.2f}".format(f1_score2))
print("ROC AUC score for classification_region2: {:.2f}".format(roc_auc2))

print("\nAccuracy for classification_region3: {:.2f}".format(accuracy3))
print("F1 score for classification_region3: {:.2f}".format(f1_score3))
print("ROC AUC score for classification_region3: {:.2f}".format(roc_auc3))

Accuracy for classification_region1: 0.76
F1 score for classification_region1: 0.36
ROC AUC score for classification_region1: 0.60

Accuracy for classification_region2: 0.83
F1 score for classification_region2: 0.11
ROC AUC score for classification_region2: 0.53

Accuracy for classification_region3: 0.80
F1 score for classification_region3: 0.53
ROC AUC score for classification_region3: 0.69
